In [3]:
from IPython.parallel import Client
c = Client(profile='default')

/home/robert/software/anaconda3/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@rmporsch')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [4]:
import os
ar = c[:].apply_async(os.getpid)
pid_map = ar.get_dict()

In [5]:
pid_map

{0: 19019, 1: 19020}

In [6]:
%px import socket
%px print("hosts:", socket.gethostname())

[stdout:0] hosts: rmporsch
[stdout:1] hosts: rmporsch


In [7]:
squares = c[:].map_sync(lambda x: x**2, range(32))
print("squares:", squares)

squares: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961]


In [11]:
%%px

from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
print(rank)
print('data')
# passing MPI datatypes explicitly
if rank == 0:
    data = numpy.arange(100, dtype='i')
    numpy.random.shuffle(data)
    print(data)
    comm.Send([data, MPI.INT], dest=1, tag=77)
    print("{0}: sent data to 1: {1}".format(rank, data))
elif rank == 1:
    data = numpy.empty(100, dtype='i')
    comm.Recv([data, MPI.INT], source=0, tag=77)
    print("{0}: received data from 0: {1}".format(rank, data))
else:
    print("{0}: idle".format(rank))

CompositeError: one or more exceptions from call to method: execute
[0:execute]: Exception: Invalid rank, error stack:
MPI_Send(173): MPI_Send(buf=0x557f97dd9280, count=100, MPI_INT, dest=1, tag=77, MPI_COMM_WORLD) failed
MPI_Send(98).: Invalid rank has value 1 but must be nonnegative and less than 1
[1:execute]: Exception: Invalid rank, error stack:
MPI_Send(173): MPI_Send(buf=0x5607e281f280, count=100, MPI_INT, dest=1, tag=77, MPI_COMM_WORLD) failed
MPI_Send(98).: Invalid rank has value 1 but must be nonnegative and less than 1